In [12]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torch
import torchvision

import os
import numpy as np

from MNISTClassifier import*

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True

## Loading the datasets

In [13]:
n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5


train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('../../datasets/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('../../datasets/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=1, shuffle=True)

testset = np.load("sample_test_set_data.npy")
testset_labels = np.load("sample_test_set_labels.npy")

## Training models

In [14]:
model_names = ['model_1', 'model_2', 'model_3', 'model_4']
model_params = [[4,2], [2,1], [8,4], [16,8]]

for model_name, model_param in zip(model_names, model_params):
    
    print("---------------------{}----------------------".format(model_name))
    
    print("Params:{}".format(model_param))

    network = Net(model_param)
    optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                          momentum=momentum)

    if not os.path.exists('results/{}'.format(model_name)):
        os.makedirs('results/{}'.format(model_name))

    for epoch in range(1, n_epochs + 1):
        train(epoch, network.cuda(), optimizer, model_name, train_loader)
        test(network.cuda(), test_loader)

    test_model = load_model(model_name)
    test_sample(test_model.cuda(), testset, testset_labels)

---------------------model_1----------------------
Params:[4, 2]
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.304732
Train Epoch: 1 [32000/60000 (53%)]	Loss: 1.934361

Test set: Avg. loss: 1.8736, Accuracy: 2253/10000 (23%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 1.894111
Train Epoch: 2 [32000/60000 (53%)]	Loss: 1.690367

Test set: Avg. loss: 1.7377, Accuracy: 3035/10000 (30%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 1.705384
Train Epoch: 3 [32000/60000 (53%)]	Loss: 1.627140

Test set: Avg. loss: 1.6482, Accuracy: 3265/10000 (33%)


Test set: Avg. loss: 81.2569, Accuracy: 15/50 (30%)

---------------------model_2----------------------
Params:[2, 1]
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.405226
Train Epoch: 1 [32000/60000 (53%)]	Loss: 2.037874

Test set: Avg. loss: 1.8831, Accuracy: 2440/10000 (24%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 1.849847
Train Epoch: 2 [32000/60000 (53%)]	Loss: 1.777777

Test set: Avg. loss: 1.7148, Accuracy: 3386/10000 (34%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 1.653205
Train 

## Loading the trained models

In [15]:
#THis is how models should be used outside the training environment

model_names = ['model_1', 'model_2', 'model_3', 'model_4']
model_params = [[4,2], [2,1], [8,4], [16,8]]

for model_name, model_param in zip(model_names, model_params):
    
    print("---------------------{}----------------------".format(model_name))
    
    print("Params:{}".format(model_param))

    test_model = load_model(model_name)
    losses, sim_log = test_sample(test_model.cuda(), testset, testset_labels)
#Losses are numerical losses per sample. Sim log is a log of successes and failures.
#Both metrics can be used to reward the adverserial for its decisions.

---------------------model_1----------------------
Params:[4, 2]

Test set: Avg. loss: 59.8022, Accuracy: 30/50 (60%)

---------------------model_2----------------------
Params:[2, 1]

Test set: Avg. loss: 80.9982, Accuracy: 17/50 (34%)

---------------------model_3----------------------
Params:[8, 4]

Test set: Avg. loss: 33.5780, Accuracy: 39/50 (78%)

---------------------model_4----------------------
Params:[16, 8]

Test set: Avg. loss: 21.9500, Accuracy: 42/50 (84%)



In [16]:
testset.shape

(50, 1, 1, 28, 28)